# Import package

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
from sklearn.model_selection import train_test_split

# Read in training set

In [2]:


def read_trainData(filename):
    # 讀入檔案
    raw_data = pd.read_csv(filename, 
                            header=None, 
                            encoding='utf8')

    # 去除標頭和多餘直行
    row, col = raw_data.shape
    raw_data  = raw_data.iloc[1:row, 3:col]              

    # 去除多餘空格
    raw_data = raw_data.replace(r'\s+', '', regex=True)  

    # 將特殊符號替換為 0
    special_chars = ['#', '*', 'x', 'A']  # 定義需要替換的特殊符號
    raw_data.replace(special_chars, 0.0, inplace=True)

    # 轉換成numpy & 浮點數型別
    data = raw_data.values 
    data = data.astype('float')

    X, Y = [], []

    # 起始值:0 , 結束值: data.shape[0], 每次增加步長: 18*20
    for month in range(0, data.shape[0], 18*20):
        # month: 第幾月份
        days = np.vsplit(data[month:month+18*20], 20) # shape: (18*24) *20 -> 數據按照行數分割成 20 天
        concat = np.concatenate(days, axis=1) # shape: (18 feature, 480(days*hr)) -> 將20天數據沿著水平方向拼接
        # print(concat.shape)
        for hour in range(0, concat.shape[1]):
            # hour: 第幾小時    
            features = concat[:, hour:hour+1].flatten() # 選取從第 j 小時到第 j+N 小時的數據
            features = np.append(features, [1])   # 特徵向量的末尾添加一個 1，這是為了引入偏置項 w0     
            # print(features)
            X.append(features)
            Y.append([concat[9, hour]])             # 第 9 行（是feature PM2.5），目標值

        # if month == 0:  
        #     print(f"np.array(X): {np.array(X)}")
        #     # print(X)


    X = np.array(X)
    Y = np.array(Y)

    print(f"X.shape: {X.shape}")
    print(f"Y.shape: {Y.shape}")
    
    return X, Y

# Read in Testing set

In [3]:

def read_testData(filename):
    # 讀入檔案
    raw_data = pd.read_csv(filename, 
                            header=None, 
                            encoding='utf8')

    # 去除標頭和多餘直行
    row, col = raw_data.shape
    raw_data  = raw_data.iloc[:, 2:col]              

    # 去除多餘空格
    raw_data = raw_data.replace(r'\s+', '', regex=True)  

    # 將特殊符號替換為 0
    special_chars = ['#', '*', 'x', 'A','WIND_DIR+D2070EC']  # 定義需要替換的特殊符號
    raw_data.replace(special_chars, 0.0, inplace=True)
    
    # 轉換成numpy & 浮點數型別
    data = raw_data.values 
    data = data.astype('float')
    # print(data.shape)
    # 每個月剩餘的天數
    test_X  = []

    for month in range(0, data.shape[0], 18*1):
        # i : 第幾個月
        day = data[month:month+18*1]             # shape: (18 feature, (1day*9hr))

        for hour in range(0, day.shape[1]):
            features = day[:, hour:hour+1].flatten() 
            # print(features)
            features = np.append(features, [1])
            test_X.append(features)

        # if month == 0:  
        #     print(f"np.array(X): {np.array(test_X).shape}")


    test_X = np.array(test_X)    

    print(f"test_X.shape: {test_X.shape}")
    
    return test_X

In [4]:
test_X = read_testData('input_data/test.csv')
print(test_X[0])

test_X.shape: (2196, 19)
[ 18.2    2.41   0.77   0.29   6.8   30.9   37.7    4.1   53.    35.
   0.    84.     2.8    2.7  140.   120.     0.4    0.5    1.  ]


# Linear Regression

In [5]:
class Linear_Regression(object):
    def __init__(self):
        pass
    def train(self, train_X, train_Y):
        '''
        普通最小二乘法（Ordinary Least Squares, OLS），
        它是解線性回歸問題的常用方法。
        最小二乘法的目標是最小化預測值與真實值之間的誤差平方和，
        並通過矩陣運算來求解最優的權重 W
            np.matmul() 是 NumPy 中用來進行矩陣乘法（Matrix Multiplication）的函數
        
            W = (X^T * X)^-1 *  X^T y
        '''
        
        # X^T X: 計算 X 的轉置與 X 的矩陣乘積
        X_transpose_X = np.matmul(np.transpose(train_X), train_X)
        # (X^T X)^-1: 計算 (X^T X) 的逆矩陣
        X_transpose_X_inv = np.linalg.inv(X_transpose_X)
        # X^T y: 計算 X 的轉置與 y 的矩陣乘積
        X_transpose_Y = np.matmul(np.transpose(train_X), train_Y)
        # W = (X^T X)^-1 X^T y: 計算權重 W
        W = np.matmul(X_transpose_X_inv, X_transpose_Y)
        # 保存計算得到的 W，方便後續使用進行預測
        self.W = W
        
    def predict(self, test_X):
        '''
            predict_Y = X*W
        '''
        predict_Y = np.matmul(test_X, self.W)
        return predict_Y 
    


In [6]:
def RMSE(predict_Y, real_Y):
    N = len(predict_Y)  # 樣本數量
    loss = np.sqrt(np.sum((predict_Y - real_Y)**2) / N)
    return loss

In [7]:
def plot_RMSE(valid_loss, train_loss, filename):
    # assert len(train_set_loss) == len(valid_loss)
    length = len(train_loss)
    plt.figure(figsize = (12, 8))
    plt.xticks(range(1, length + 1))
    plt.plot(range(1, length+1), train_loss, 'b', label = 'train loss')
    plt.plot(range(1, length+1), valid_loss, 'r', label = 'test loss')
    plt.legend()
    plt.xlabel('N')
    plt.ylabel('RMSE loss')
    plt.show()
    plt.savefig(filename + '.png')

In [8]:
def Write(answer):
    with open("resultA.csv", "w", newline='') as f:
        w = csv.writer(f)
        title = ['index','answer']
        w.writerow(title) 
        for i in range(244):
            content = ['index_'+str(i),answer[i][0]]
            w.writerow(content) 
            

In [9]:
def filter(data, del_features):
    features = ['AMB', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10', 'PM2.5', 'RAINFALL', 'RH', 'SO2', 'THC', 'WD_HR', 'WIND_DIR', 'WIND_SPEED', 'WS_HR']
    del_pos = []
    for i in del_features:
        for j in range(len(features)):
            if(i == features[j]):
                del_pos.append(j)
    return np.delete(data, del_pos, 1)

# Pearson product-moment correlation coefficient
# X: feature	Y: PM2.5
def Pearson(X, Y):
    N = len(X)
    mu_X = np.sum(X)/N
    mu_Y = np.sum(Y)/N

    Deno = 0
    for i in range(N):
        Deno += (X[i]-mu_X)*(Y[i]-mu_Y)

    sigma_X = 0
    sigma_Y = 0
    for i in range(N):
        sigma_X += (X[i]-mu_X)**2
        sigma_Y += (Y[i]-mu_Y)**2
    sigma_X = np.sqrt(sigma_X)
    sigma_Y = np.sqrt(sigma_Y)

    return Deno/(sigma_X*sigma_Y)

In [10]:
def Write_Ret(features, numbers):
    with open('Relation/feature.txt', "w", newline='') as f:
        for i in range(len(numbers)):
           f.write('{} Pearson number: {}\n'.format(features[i], numbers[i]))

In [11]:
def plot_relation(train_X, train_Y, features):
    Ret_Pearson = []
    Ret_feature = []
    N = len(features)
    length = len(train_X)
    X = np.hsplit(train_X, len(train_X[0]))
    Y = train_Y.flatten('F')
    for i in range(N):
        filename = 'Relation/{}-relation.png'.format(features[i])
        plt.figure(figsize = (12, 8))
        plt.xticks(range(1, length+1))
        plt.plot(range(1, length+1), X[i].flatten('F'), 'b', label = features[i])
        plt.plot(range(1, length+1), Y, 'r', label = 'PM2.5 Next hour')
        plt.legend()
        plt.xlabel('N')
        plt.savefig(filename)
        plt.show()
        Pearson_number = Pearson(X[i], Y)
        Ret_Pearson.append(Pearson_number)
        Ret_feature.append(features[i])
        print('{} Pearson number: {}'.format(features[i], Pearson_number))
    
    Write_Ret(Ret_feature, Ret_Pearson)
    return Ret_Pearson

In [12]:
attrs = ['AMB', 'CH4', 'CO', 'NMHC', 'NO', 'NO2',
        'NOx', 'O3', 'PM10', 'PM2.5', 'RAINFALL', 'RH',
        'SO2', 'THC', 'WD_HR', 'WIND_DIR', 'WIND_SPEED', 'WS_HR']


if __name__ == '__main__' :
    filename_train = 'input_data/train.csv'
    filename_test = 'input_data/test.csv'
    
    features = ['AMB', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10', 'PM2.5', 'RAINFALL', 'RH', 'SO2', 'THC', 'WD_HR', 'WIND_DIR', 'WIND_SPEED', 'WS_HR']
    del_features = []

    train_set_loss = []
    test_set_loss = []

    train_set_loss_filtered = []
    test_set_loss_filtered = []

    X, Y = read_trainData(filename_train)
    test_X = read_testData(filename_test)
    train_X, valid_X, train_Y, valid_Y = train_test_split(X, Y, test_size=0.2, random_state=42)
        
    # Ret = plot_relation(train_X, train_Y, features)
    # min_R = 0.3
    # for i in range(len(Ret)):
    #     if(np.absolute(Ret[i]) < min_R):
    #         del_features.append(features[i])
    # print(del_features)        
    del_features = ['AMB', 'CH4', 'NO', 'O3', 'RAINFALL', 'RH', 'SO2', 'THC', 'WD_HR', 'WIND_DIR', 'WIND_SPEED', 'WS_HR']

    # Filtered
    train_X = filter(train_X, del_features)
    valid_X = filter(valid_X, del_features)
    test_X = filter(test_X, del_features)
    

    model = Linear_Regression()
    model.train(train_X, train_Y)

    predict_Y = model.predict(valid_X)
    valid_loss = RMSE(predict_Y, valid_Y)
    
    predict_Y = model.predict(train_X)
    train_loss = RMSE(predict_Y, train_Y)

    predict_result = model.predict(test_X)
    Write(predict_result)
    # train_set_loss.append(train_loss)
    # test_set_loss.append(test_loss)
    # test_set_loss = []
    # plot_RMSE(valid_loss, train_loss, 'Normal_loss')
    print(valid_loss)
    print(train_loss)


X.shape: (5760, 19)
Y.shape: (5760, 1)
test_X.shape: (2196, 19)
4.25775371871331e-14
4.2873102710161814e-14
